<a href="https://colab.research.google.com/github/hchristiaan/TrainCNN4SignatureVerification/blob/master/ProjectCode_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Please note that:
#     # - Means comment explaining the code
#     ### - Means things that need to be addressed

In [0]:
# Import all required repositories for the project
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2

import keras
import keras.backend as K
import tensorflow as tf
from keras import applications
from keras.models import Model
from keras.layers import Flatten, Dense, Input,concatenate
from keras.optimizers import Adam
from keras.models import load_model, model_from_json

import random

In [0]:
# Import the google drive to upload the data
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# Unzip the image dataset
!unzip '/content/drive/Team Drives/inputdataset/UTSig.zip' -d /content

In [0]:
# File paths of the image data used later in the project
gentr="/content/....."
forgtr="/content/....."

gent="/content/..."
forgt="/content/..."

In [0]:
# Code block for working with the image files (resizing, retreiving files, etc.)

# Standard height weight for images
img_width, img_height, channels = 224, 224, 3

dim = (img_width, img_height)

### Do we need this??? ###
def to_rgb(img):
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA) 
    img_rgb = np.asarray(np.dstack((img, img, img)), dtype=np.uint8)
    return img_rgb


# Takes in file patch and image name and returns resized image
def returnimages(path,img):
    image=cv2.imread(path+"/"+ img)                  #bringing the image
    image=cv2.resize(image, (img_width, img_height))
    image=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image=to_rgb(image).reshape(1,img_width, img_height,3)/255.0       #resizing and normalizing    
    return image     

# Takes in number of author, path for genuine and forged signatures
# Returns one image name (m.pop()), and two arrays of file names n = forgary, m = genuine
# Used for generator()
def getfiles(num,gen,forg):
    a=os.listdir(gen)
    b=os.listdir(forg)
    c=str(num)
    c=c[2:]
    if(len(c)==2):
        c=c+"0"
        
### This code needs to be altered ###  
    n,m=[],[]
    for i in b:
        if i.endswith(c+".png"):
            n=n+[i]
        elif i.endswith(c+".PNG"):
            n=n+[i]
    for i in a:
        if i.endswith(c+".png"):
            m=m+[i]
        elif i.endswith(c+".PNG"):
            m=m+[i]
    return m.pop(),n,m
######################################

### Needs to be looked into ###
### used for generator2()   ###
def getfiles2(num):
    a=os.listdir(gentr)
    b=os.listdir(forgtr)
    c=str(num)
    c=c[2:]
    if(len(c)==2):
        c=c+"0"
### Needs to be altered ###
    n,m=[],[]
    for i in b:
        if (i.endswith(c+"_001_6g.png") or i.endswith(c+"_002_6g.png") or i.endswith(c+"_003_6g.png")
            or i.endswith(c+"_004_6g.png") or i.endswith(c+"_005_6g.png")):
            n=n+[i]
        elif (i.endswith(c+"_001_6g.PNG") or i.endswith(c+"_002_6g.PNG") or i.endswith(c+"_003_6g.PNG")
              or i.endswith(c+"_004_6g.PNG") or i.endswith(c+"_005_6g.PNG")):
            n=n+[i]
    for i in a:
        if (i.endswith(c+"_001_6g.png") or i.endswith(c+"_002_6g.png") or i.endswith(c+"_003_6g.png")
            or i.endswith(c+"_004_6g.png") or i.endswith(c+"_005_6g.png")):
            m=m+[i]
        elif (i.endswith(c+"_001_6g.PNG") or i.endswith(c+"_002_6g.PNG") or i.endswith(c+"_003_6g.PNG")
              or i.endswith(c+"_004_6g.PNG") or i.endswith(c+"_005_6g.PNG")):
            m=m+[i]
    return m.pop(),n,m
#################################

In [0]:
# Function for defining loss variable in model, leave for now, look into altering?
# Unsure what y_pred is doing, or how this works....
def triplet_loss(y_true, y_pred):
    alpha = 0.5
    anchor, positive, negative =y_pred[0,0:512], y_pred[0,512:1024], y_pred[0,1024:1536]
    
    positive_distance = K.mean(K.square(anchor - positive),axis=-1)
    negative_distance = K.mean(K.square(anchor - negative),axis=-1)
    return K.mean(K.maximum(0.0, positive_distance - negative_distance + alpha))

In [0]:
# Defining the architecture. chose to go with 16 as it is smaller...
model = applications.vgg16.VGG16(weights='imagenet', include_top=False, pooling='max')

### Imput baseline model ### Unsure how this is done, commenting for now
#basemodel = applications.vgg16.VGG16(weights='imagenet', include_top=False, pooling='max')

In [0]:
# For layers from 1 to 15 freeze the weights
### We will need to do a test for each layer to find the optimal transfer* ###
### For now just copying over the code from example
for layer in model.layers[:15]:
    layer.trainable = False